In [ ]:
--1
select  *
from    sports
order by name;

In [ ]:
--2 
select  distinct S.name
from    results R 
JOIN    sports S on R.sportid = S.id
WHERE   R.result is not NULL;

In [ ]:
--3
select  count(distinct peopleid)
from    results;

In [ ]:
--4
select  id, name 
from    people
where   id in ( 
    select  peopleid as id 
    from    results
    GROUP BY peopleid 
    having  count(*) >= 20 
);

In [ ]:
select  id, name 
from    people P
join    results R
on      P.id = R.peopleid
group by P.id 
having  count(*) >= 20;

In [ ]:
create or replace view has_gt_20_results as 
select  peopleid
FROM    (select * from results where result is not null) r
group by peopleid
having count(*) >= 20;

select  id, name 
from    people P
join    has_gt_20_results R 
on      R.peopleid = P.id;

In [ ]:
--5
select  P.id, P.name, (select description from gender where gender = P.gender)
from    results R 
join    sports S 
ON      R.sportid = S.id AND R.result = S.record
join    people P 
ON      P.id = R.peopleid;

In [ ]:
--6
select  S.name, count(distinct R.peopleid) as numathletes
from    sports S 
join    results R
on      S.record = R.result and S.id = R.sportid and S.record is not null
group by S.name;

In [ ]:
--7
select  
        P.id, 
        P.name, 
        max(R.result) as best, 
        to_char(
            (select record from sports where name like 'Triple Jump') - max(R.result)
            , '0D99'
         ) as difference
from    results R 
JOIN    people P 
ON      R.peopleid = P.id and R.sportid = (select id from sports where name like 'Triple Jump')
        and R.result is not null
GROUP BY (P.id,P.name)
HAVING  count(*) >= 20;

In [ ]:
--8
select  distinct P.id, P.name, G.description
from    competitions C 
join    results R on C.place = 'Hvide Sande' and extract(year from C.held) = '2009'
        and R.competitionid = C.id
join    people P ON P.id = R.peopleid
join    gender G ON G.gender = P.gender;

In [ ]:
--9
select  P.name, G.description
FROM    people p
join    gender g on p.gender = g.gender
where   name like '% J%sen';

In [ ]:
--10
select P.name, S.name, 
    case when R.result is null or S.record is null then 
            null 
        else 
            cast(100*R.result/S.record as int) || '%'
    end as percentage
from    results R 
join    people P on R.peopleid = P.id 
join    sports S on R.sportid = S.id;

In [ ]:
select count(*) from results;

In [ ]:
--11
select  count(distinct R.peopleid)
from    results R 
WHERE   R.sportid is null or R.competitionid is null or R.result is null;

In [ ]:
--12
select  S.id, S.name, cast(max(R.result) as float) as maxres
FROM    sports S 
join    results R on R.sportid = S.id 
group by (S.id,S.name)
order by S.id;

In [ ]:
--13
select  R.peopleid, P.name, count(R.result)
from    results R
JOIN    sports S on R.sportid = S.id and R.result = S.record and S.record is not null
JOIN    people P on P.id = R.peopleid
group by R.peopleid, P.name
having  count(distinct sportid) >= 2;

In [ ]:
--14
create or replace view max_results as
select  S.id as sportid, S.name as name, S.record as record, max(R.result) as result
from    Results R 
JOIN    Sports S on S.id = R.sportid and R.result is not null
group BY S.id, S.name, S.record;

select  distinct P.id, P.name, P.height, R.result, MR.name, 
    case when MR.record = R.result then 'yes' else 'no' end as "record?"
from    max_results MR 
JOIN    results R on MR.sportid = R.sportid and MR.result = R.result
JOIN    people P on P.id = R.peopleid;

In [ ]:
--15
select  P.id, P.name
FROM    people P 
except 
select  P.id, P.name 
FROM    Results R 
JOIN    People P on R.peopleid = P.id and R.competitionid is not null;

In [ ]:
--16
select  P.id, P.name
from    competitions C
join    results R on R.competitionid = C.id and extract(year from C.held) = '2002' and extract(month from C.held) = '06'
join    people P on R.peopleid = P.id
union
select  P.id, P.name 
from    sports S 
join    Results R on S.id = R.sportid and S.name = 'High Jump' and S.record = R.result
JOIN    People P on P.id = R.peopleid;

In [ ]:
--17
select  P.id, P.name
from    Results R
join    people P on R.peopleid = P.id 
group by P.id 
HAVING  count(distinct R.sportid) = 1 
        and max(R.result) = (select S.record from sports S where S.id = max(R.sportid));

In [ ]:
--18
select count(*) from (
    select  count(*)
    from    competitions C
    join    results R on R.competitionid = C.id
    group by R.peopleid 
    having  count(distinct C.place) >= 10
) tmp;

In [ ]:
--19
select  P.id, P.name
FROM    Results R 
join    sports S on S.id = R.sportid and S.record = R.result
join    people P on R.peopleid = P.id
group by P.id, P.name
having  count(distinct S.id) = (select count(distinct id) from Sports);

In [ ]:
--20
select  S.id, S.name, S.record, min(R.result) as worst
from    Sports S
JOIN    Results R on R.sportid = S.id 
join    competitions C on C.id = R.competitionid
group by S.id, S.name, S.record
having  count(distinct C.place) = (select count(distinct place) from competitions);